<a href="https://colab.research.google.com/github/dakilaledesma/FSUS_Explanations/blob/main/notebooks/test/S2D_O600G4_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%time
! unzip -q drive/MyDrive/UNC/S2022P/GBIF_O_21_fourth.zip -d /content/temp
! mv /content/temp/content/dataset/test /content/test
! rm -rf /content/temp

CPU times: user 49 ms, sys: 24.4 ms, total: 73.4 ms
Wall time: 5.97 s


In [ ]:
! git clone https://github.com/rwightman/pytorch-image-models.git

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 10805, done.
remote: Counting objects: 100% (332/332), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 10805 (delta 186), reused 245 (delta 154), pack-reused 10473
Receiving objects: 100% (10805/10805), 20.45 MiB | 20.57 MiB/s, done.
Resolving deltas: 100% (7898/7898), done.


In [ ]:
! python -u ./pytorch-image-models/inference.py test -b 128 --input-size 3 600 600 --num-classes 185 --no-test-pool --checkpoint /content/drive/MyDrive/UNC/S2022P/output/20220816-163320-sequencer2d_l-600/last.pth.tar --model sequencer2d_l

Loaded state_dict from checkpoint '/content/drive/MyDrive/UNC/S2022P/output/20220816-163320-sequencer2d_l-600/last.pth.tar'
Model sequencer2d_l created, param count: 53984441
Predict: [0/7] Time 16.394 (16.394)


In [ ]:
from glob import glob
import os

class_dict = {}
for fn in glob("test/**/*.*", recursive=True):
  cat = int(fn.split("/")[-2])
  bn = os.path.basename(fn)

  class_dict[bn] = cat

In [ ]:
def get_acc():
  output_file = open("topk_ids.csv")
  output_lines = output_file.readlines()

  corr = 0
  total = 0
  for line in output_lines:
    
    line = line.strip().split(",")
    cat = class_dict[line[0]]
    pred = int(line[1])

    if cat == pred:
      corr += 1

    total += 1

  print(corr / total)

In [ ]:
# S2D O600G4 Epoch 80/80
get_acc()

0.6254458977407847
